In [1]:
import pyreadstat as prs
from helper import *
import os

In [2]:
files = ['data/morg79.dta','data/morg19.dta' ]
# for each year, log hourly wage, % < high school, % with college, years edu, potential exp, potential exp squared, sample size
# do this for 3 criteria: men and women age 24-65, men age 24-65, women age 24-65
# weight based on hrearnwt = dataframe['earnwt'] * dataframe['uhours']

# define dictionary of dataframes
dfs = {}

for file in files:
    print(file)
    # create dataframe
    df, meta = prs.read_dta(file)
    # create year variable
    # split .dta, take last two characters, convert to int
    year = int(file.split('.')[0][-2:])
    dfs[year] = df

print(dfs.keys())

data/morg79.dta
data/morg19.dta
dict_keys([79, 19])


In [3]:
inflation = [[33.804,34.728,35.590,36.451], [102.877, 103.422, 103.674, 104.080]]
inflation_2019 = [102.877, 103.422, 103.674, 104.080]
# now that data is loaded in dictionary, can create tables
# add to each df the new variables
for j, year in enumerate(dfs.keys()):
    df = dfs[year]
    # create new variables

    # make sure to compare the right row entries so the division makes sense
    eps = 1e-6
    if 'uhours' in df.columns:
        df['uhours'] = pd.to_numeric(df['uhours'], errors='coerce').fillna(0)
        df['earnwke'] = pd.to_numeric(df['earnwke'], errors='coerce').fillna(0)
        # get as np array
        uhours = df['uhours'].to_numpy()
        df['hrwage'] = np.where(uhours > eps, df['earnwke'] / uhours, 0)
        dfs[year]['hrearnwt'] = dfs[year]['earnwt'] * dfs[year]['uhours']
    elif 'uhourse' in df.columns:
        df['uhourse'] = pd.to_numeric(df['uhourse'], errors='coerce').fillna(0)
        # get as np array
        uhours = df['uhourse'].to_numpy()
        df['earnwke'] = pd.to_numeric(df['earnwke'], errors='coerce').fillna(0)
        df['hrwage'] = np.where(uhours > eps, df['earnwke'] / uhours, 0)
        dfs[year]['hrearnwt'] = dfs[year]['earnwt'] * uhours
    
    ## first adjust for inflation for each quarter ##
    # get the row from the inflation list
    inflation_row = inflation[j]
    realhrwage = df['hrwage'] 
    # if month between 1 and 3, then multiply by 100/33.804
    realhrwage = np.where(df['intmonth'] <= 3, realhrwage * 100/inflation_row[0], realhrwage)
    # if month between 4 and 6, then multiply by 100/34.728
    realhrwage = np.where((df['intmonth'] > 3) & (df['intmonth'] <= 6), realhrwage * inflation_2019[0]/inflation_row[1], realhrwage)
    # if month between 7 and 9, then multiply by 100/35.590
    realhrwage = np.where((df['intmonth'] > 6) & (df['intmonth'] <= 9), realhrwage * inflation_2019[1]/inflation_row[2], realhrwage)
    # if month between 10 and 12, then multiply by 100/36.451
    realhrwage = np.where((df['intmonth'] > 9) & (df['intmonth'] <= 12), realhrwage * inflation_2019[2]/inflation_row[3], realhrwage)
    
    df['realhrwage'] = realhrwage
    # where realhrwage <= 2 or >= 250, set to 0
    df['realhrwage'] = np.where(df['realhrwage'] <= 2, 0, df['realhrwage'])
    df['realhrwage'] = np.where(df['realhrwage'] >= 250, 0, df['realhrwage'])


    ## get educ; treat gradeat vs grade92 separately ##
    # if grade32 exists, then need to convert that; otheerwise have to deal with gradeat and gradecp
    # create educ variable
    if 'grade92' in df.columns:
        df['educ'] = df['grade92']
        
        df.loc[df['grade92'] == 31, 'educ'] = 0
        df.loc[df['grade92'] == 32, 'educ'] = 2.5
        df.loc[df['grade92'] == 33, 'educ'] = 5.5
        df.loc[df['grade92'] == 34, 'educ'] = 7.5
        df.loc[df['grade92'] == 35, 'educ'] = 9
        df.loc[df['grade92'] == 36, 'educ'] = 10
        df.loc[df['grade92'] == 37, 'educ'] = 11
        df.loc[df['grade92'] == 38, 'educ'] = 12
        df.loc[df['grade92'] == 39, 'educ'] = 12
        df.loc[df['grade92'] == 40, 'educ'] = 13
        df.loc[df['grade92'] == 41, 'educ'] = 14
        df.loc[df['grade92'] == 42, 'educ'] = 14
        df.loc[df['grade92'] == 43, 'educ'] = 16
        df.loc[df['grade92'] == 44, 'educ'] = 18
        df.loc[df['grade92'] == 45, 'educ'] = 18
        df.loc[df['grade92'] == 46, 'educ'] = 18

    else:
        df['educ'] = df['gradeat']
        # if gradecp is 0, then subtract 1
        df.loc[df['gradecp'] == 0, 'educ'] = df['educ'] - 1
    

    df['logwage'] = np.log(df['realhrwage'].astype(float))
    df['logwage'].replace(-np.inf, np.nan, inplace=True)  # Replace -inf with NaN or another placeholder

    df['hrearnwt'] = np.where(df['logwage'].notna(), df['hrearnwt'], 0)
   
   
    df['lths'] = dfs[year]['educ'] < 12
    df['college'] = dfs[year]['educ'] >= 16
    df['exp'] = dfs[year]['age'] - dfs[year]['educ'] - 6
    df['exp2'] = dfs[year]['exp']**2

    # reset key value
    dfs[year] = df

/Users/oscarscholin/anaconda3/lib/python3.11/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/var/folders/zx/ck7rkn5j0_zfqqnb92k3668r0000gn/T/ipykernel_20301/3680371078.py:75: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['logwage'].replace(-np.inf, np.nan, inplace=True)  # Replace -inf with NaN or another placeholder
/var/folders/zx/ck7rkn5j0_zfqqnb92k3668r0000gn/T/ipykernel_20301/3680371078.py:52: FutureWarning: Setting an i

In [4]:
combined_df = pd.concat([dfs[79], dfs[19]], ignore_index=True)
fulltime = combined_df['uhourse'] >= 35
combined_df['fulltime'] = fulltime

In [5]:
combined_df

,minsamp,intmonth,hhid,state,smsarank,hhnum,activlwr,hourslw,reasonlw,absentlw,...,ym,ch02,ch35,ch613,ch1417,ch05,ihigrdc,docc00,dind02,fulltime
0,4,1,003001221503,93.0,0.0,1,1.0,40,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
1,4,1,003001221503,93.0,0.0,1,1.0,40,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
2,4,1,003001221503,93.0,0.0,1,5.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,4,1,003001222503,93.0,0.0,1,7.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,4,1,003001222503,93.0,0.0,1,1.0,40,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
619791,4,12,802505349610555,NaN,NaN,1,NaN,32,NaN,NaN,...,716.0,0.0,0.0,0.0,0.0,0.0,12.0,21,47,False
619792,4,12,802505349610555,NaN,NaN,1,NaN,40,NaN,NaN,...,716.0,0.0,0.0,0.0,0.0,0.0,NaN,20,21,True
619793,4,12,876944601471509,NaN,NaN,1,NaN,NaN,NaN,NaN,...,716.0,0.0,0.0,0.0,0.0,0.0,14.0,22,23,False
619794,4,12,905527030610215,NaN,NaN,1,NaN,40,NaN,NaN,...,716.0,0.0,0.0,0.0,0.0,0.0,13.0,19,4,True


In [6]:
combined_df['docc00']

0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
         ... 
619791     21
619792     20
619793     22
619794     19
619795     10
Name: docc00, Length: 619796, dtype: object

In [7]:
# Ensure boolean conditions for 'college' and 'fulltime'
combined_df['college'] = combined_df['college'].astype(bool)
combined_df['fulltime'] = combined_df['fulltime'].astype(bool)

# Define conditions
collgrad_fulltime = combined_df['college'] & combined_df['fulltime']
collgrad_parttime = combined_df['college'] & ~combined_df['fulltime']
nocollgrad_fulltime = ~combined_df['college'] & combined_df['fulltime']
nocollgrad_parttime = ~combined_df['college'] & ~combined_df['fulltime']

# Function to calculate weighted proportion
def weighted_proportion(condition, sex, weights):
    relevant_weights = weights[(condition) & (combined_df['sex'] == sex)]
    total_weights = np.sum(weights[combined_df['sex'] == sex])
    return np.sum(relevant_weights) / total_weights if total_weights > 0 else 0

# Calculate proportions for males and females
male_proportions = [weighted_proportion(cond, 1, combined_df['hrearnwt']) for cond in 
                    [collgrad_fulltime, collgrad_parttime, nocollgrad_fulltime, nocollgrad_parttime]]
female_proportions = [weighted_proportion(cond, 2, combined_df['hrearnwt']) for cond in 
                      [collgrad_fulltime, collgrad_parttime, nocollgrad_fulltime, nocollgrad_parttime]]

# Calculate adjustment factors (male proportion / female proportion)
adjustment_factors = [male / female if female > 0 else 1 for male, female in zip(male_proportions, female_proportions)]

# create new column for the dfl weights
combined_df['dflweight'] = combined_df['hrearnwt']

# Apply adjustment factors to female weights for each condition
for i, cond in enumerate([collgrad_fulltime, collgrad_parttime, nocollgrad_fulltime, nocollgrad_parttime]):
    combined_df.loc[cond & (combined_df['sex'] == 2), 'dflweight'] *= adjustment_factors[i]

# Verify the adjustment by recalculating the proportions
adjusted_female_proportions = [weighted_proportion(cond, 2, combined_df['dflweight']) for cond in 
                               [collgrad_fulltime, collgrad_parttime, nocollgrad_fulltime, nocollgrad_parttime]]

# Now, adjusted_female_proportions should be closer to male_proportions

In [8]:
male_proportions

[0.29931287477303253,
 0.009671670311376458,
 0.6489630601764321,
 0.0420523947391591]

In [9]:
female_proportions

[0.31841439211571787,
 0.029608933261610824,
 0.5423927197855192,
 0.10958395483715201]

In [10]:
adjusted_female_proportions

[0.29931287477303264,
 0.009671670311376461,
 0.6489630601764321,
 0.04205239473915914]

In [15]:
# get weight that matches
# get no nans in logwage
wage_weight = combined_df['hrearnwt']
wage_weight = wage_weight[combined_df['logwage'].notna()]

logwage = combined_df['logwage']
logwage = logwage[combined_df['logwage'].notna()]

# summarize
summarize(logwage[combined_df['sex']==2], wage_weight[combined_df['sex']==2])

Num observations:  145604
Weighted Mean:  2.8449869809509742
Weighted Standard Deviation:  0.5632348140167462
Weighted Variance:  0.3172334557204788
1st percentile:  1.4722181139863721
5th percentile:  2.045259188206525
10th percentile:  2.145463007991118
25th percentile:  2.3324724102238177
50th percentile:  2.679686287212784
75th percentile:  3.07521637406679
90th percentile:  3.5121045824455264
95th percentile:  3.7874735924082557
99th percentile:  4.249901619825194
Skewness:  0.41837765627317647
Kurtosis:  0.5268683199161499
Min:  0.700663856254383
Max:  5.517552444217037


In [16]:
# get weight that matches
# get no nans in logwage
wage_weight = combined_df['dflweight']
wage_weight = wage_weight[combined_df['logwage'].notna()]

logwage = combined_df['logwage']
logwage = logwage[combined_df['logwage'].notna()]

# summarize
summarize(logwage[combined_df['sex']==2], wage_weight[combined_df['sex']==2])

Num observations:  145604
Weighted Mean:  2.8454586687740613
Weighted Standard Deviation:  0.5464405213714058
Weighted Variance:  0.2985972433966537
1st percentile:  1.4722181139863721
5th percentile:  2.045259188206525
10th percentile:  2.145463007991118
25th percentile:  2.3324724102238177
50th percentile:  2.679686287212784
75th percentile:  3.07521637406679
90th percentile:  3.5121045824455264
95th percentile:  3.7874735924082557
99th percentile:  4.249901619825194
Skewness:  0.41837765627317647
Kurtosis:  0.5268683199161499
Min:  0.700663856254383
Max:  5.517552444217037


In [ ]:
# X_cols = ['bin_age', 'bin_educ'] # make into bins and leave one out
X_cols = ['age', 'educ']
y_col = 'sex'
w_col = 'hrearnwt'
X, y, w, df_cond = prepare_data(combined_df, None, X_cols, y_col, w_col, return_dataframe=True)
y = np.where(y == 2, 0, 1) # male is 1, female is 0. # whatever is 0 gets reweighted

In [ ]:
w_new, Psi_x = run_DFL(X, y, w)

Weighted Logit Model Summary (GLM):
                 Generalized Linear Model Regression Results                  
Dep. Variable:                      y   No. Observations:               619796
Model:                            GLM   Df Residuals:                   619794
Model Family:                Binomial   Df Model:                            1
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:            -6.9526e+10
Date:                Sun, 25 Feb 2024   Deviance:                   1.3905e+11
Time:                        22:34:46   Pearson chi2:                 1.01e+11
No. Iterations:                     5   Pseudo R-squ. (CS):        -1.090e+144
Covariance Type:            nonrobust                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
age            0

In [ ]:
run_WLS(X, y, w)

                            WLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.005
Model:                            WLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     534.8
Date:                Sun, 25 Feb 2024   Prob (F-statistic):          8.59e-233
Time:                        22:34:46   Log-Likelihood:                   -inf
No. Observations:              619796   AIC:                               inf
Df Residuals:                  619793   BIC:                               inf
Df Model:                           2                                         
Covariance Type:                  HC1                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.7318      0.006    125.799      0.0

/Users/oscarscholin/anaconda3/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:807: RuntimeWarning: divide by zero encountered in log
  llf += 0.5 * np.sum(np.log(self.weights))


In [ ]:
summarize(w_new)

Num observations:  619796
Mean:  161860.64592335795
Median:  21002.567643390073
Standard Deviation:  223303.35316432835
Variance:  49864387534.43275
1st percentile:  0.0
5th percentile:  0.0
10th percentile:  0.0
25th percentile:  0.0
50th percentile:  21002.567643390073
75th percentile:  275321.6353081503
90th percentile:  512447.2975792452
95th percentile:  633734.077
99th percentile:  861832.4035584098
Skewness:  1.5851354457154851
Kurtosis:  2.744595763583294
Min:  0.0
Max:  3120656.1659999997


In [ ]:
summarize(w)

Num observations:  619796
Mean:  163627.08643388582
Median:  21825.2001953125
Standard Deviation:  224973.5688484167
Variance:  50613106680.39329
1st percentile:  0.0
5th percentile:  0.0
10th percentile:  0.0
25th percentile:  0.0
50th percentile:  21825.2001953125
75th percentile:  279614.8046875
90th percentile:  514066.5522
95th percentile:  636822.681625
99th percentile:  870190.2958500066
Skewness:  1.579701464311216
Kurtosis:  2.7614404747950676
Min:  0.0
Max:  3120656.1659999997


In [ ]:
combined_df['dfl_weight'] = w_new

In [ ]:
combined_df['dfl_weight']

0         282229.609375
1         254949.117075
2              0.000000
3              0.000000
4         286780.868168
              ...      
619791         0.000000
619792     58052.112000
619793         0.000000
619794     61429.452000
619795     54417.811873
Name: dfl_weight, Length: 619796, dtype: float64

In [ ]:
X, y, w = prepare_data(combined_df, (combined_df['sex']==2), ['educ'], 'realhrwage', 'hrearnwt')

In [ ]:
summarize(y, w)

Num observations:  326261
Weighted Mean:  20.334743399277826
Weighted Standard Deviation:  13.233246699352735
Weighted Variance:  175.11881820593007
1st percentile:  0.0
5th percentile:  0.0
10th percentile:  0.0
25th percentile:  0.0
50th percentile:  0.0
75th percentile:  13.36774848426655
90th percentile:  22.94409398692054
95th percentile:  31.761229429318504
99th percentile:  57.38818103885053
Skewness:  2.650730664987674
Kurtosis:  14.638643699737477
Min:  0.0
Max:  249.02478862413528


In [ ]:
X, y, w = prepare_data(combined_df, (combined_df['sex']==2), ['educ'], 'realhrwage', 'dfl_weight')

In [ ]:
summarize(y, w)

Num observations:  326261
Weighted Mean:  20.55338217193446
Weighted Standard Deviation:  13.355921766391885
Weighted Variance:  178.38064622998053
1st percentile:  0.0
5th percentile:  0.0
10th percentile:  0.0
25th percentile:  0.0
50th percentile:  0.0
75th percentile:  13.36774848426655
90th percentile:  22.94409398692054
95th percentile:  31.761229429318504
99th percentile:  57.38818103885053
Skewness:  2.650730664987674
Kurtosis:  14.638643699737477
Min:  0.0
Max:  249.02478862413528


In [ ]:
X, y, w = prepare_data(combined_df, combined_df['sex']==1, ['educ'], 'realhrwage', 'hrearnwt')

In [ ]:
summarize(y, w)

Num observations:  293535
Weighted Mean:  25.083569299126186
Weighted Standard Deviation:  14.79796511183094
Weighted Variance:  218.9797714509657
1st percentile:  0.0
5th percentile:  0.0
10th percentile:  0.0
25th percentile:  0.0
50th percentile:  10.353804283516743
75th percentile:  22.105118403778263
90th percentile:  34.2571236830284
95th percentile:  45.084876586550855
99th percentile:  70.09851570321841
Skewness:  1.6840813940376371
Kurtosis:  6.195857052186778
Min:  0.0
Max:  249.39232594478847


In [ ]:
X, y, w = prepare_data(combined_df, combined_df['sex']==1, ['educ'], 'realhrwage', 'dfl_weight')

In [ ]:
summarize(y, w)

Num observations:  293535
Weighted Mean:  25.083569299126186
Weighted Standard Deviation:  14.79796511183094
Weighted Variance:  218.9797714509657
1st percentile:  0.0
5th percentile:  0.0
10th percentile:  0.0
25th percentile:  0.0
50th percentile:  10.353804283516743
75th percentile:  22.105118403778263
90th percentile:  34.2571236830284
95th percentile:  45.084876586550855
99th percentile:  70.09851570321841
Skewness:  1.6840813940376371
Kurtosis:  6.195857052186778
Min:  0.0
Max:  249.39232594478847
